# Практическое задание к уроку 6. Нейросети в обработке текста

Самостоятельно обучить классификатор текстов на примере 20newsgroups

На примере 20 newsgroups попробовать разные параметры для сверток для классификации текстов

In [1]:
import random, re, glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from functools import lru_cache
from tqdm.notebook import tqdm

In [2]:
fields = ['newsgroup', 'document_id', 'from', 'subject', 'archive-name', 'last-modified']

def parse_file(path):
    path = path.replace('\\', '/')
    newsgroup=path.rsplit('/', 1)[1].rsplit('.', 1)[0]
    print(f"Читаем {path}, newsgroup {newsgroup}")
    res = []
    header=False
    with open(path, 'r') as fd:
        for line in fd:
            m = re.match(fr"({'|'.join(fields)}):\s*(\S.*)", line, re.I)
            if m:
                if not header:
                    res.append({ 'text': '', **dict((f, '') for f in fields) })
                    res[-1]['newsgroup'] = newsgroup
                    header = True
                res[-1][m.group(1).lower()] = m.group(2)
            else:
                if header:
                    if line.strip() and not re.match(fr"[\w\-]+:.*", line, re.I):
                        header = False
                if not header:
                    res[-1]['text'] += line
    return res

@lru_cache()
def parse_directory(path):
    return pd.DataFrame([ r for f in glob.glob(path + '/*.txt') for r in parse_file(f) ])

In [3]:
!pip install --upgrade pip

In [4]:
!pip install stop-words

In [5]:
!pip install pymorphy2

In [6]:
!pip install torch.utils #.data

In [11]:
from sklearn.datasets import fetch_20newsgroups

In [8]:
from torch.utils.data import DataLoader, Dataset

In [12]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

from collections import Counter


class Dataset(torch.utils.data.Dataset):
    
    
    def __init__(self, data_path):
        df = parse_directory(data_path)

        self.labels_i2w = df['newsgroup'].unique() 
        self.labels_w2i = dict( (w, i) for i, w in enumerate(self.labels_i2w) )
        self.labels = df['newsgroup'].apply(lambda ng: self.labels_w2i[ng]).tolist()
        
        sw = set(get_stop_words("en"))
        exclude = set(punctuation)
        morpher = MorphAnalyzer()

        def preprocess(txt):
            txt = ''.join(c for c in txt if c not in exclude)
            txt = txt.lower()
            return [morpher.parse(word)[0].normal_form for word in txt.split() if word and word not in sw] # удалить пустые сразу

        print('Подготовка текста ...')
        texts = [preprocess(txt) for txt in tqdm(df['text'].tolist())]
        
        print('Преобразование в индексы ...')
        words = [w for w, c in Counter(w for txt in texts for w in txt).items() if c > 10]
        
        self.words_i2w = ['', '__UNK__'] + words
        self.words_w2i = w2i = dict( (w, i) for i, w in enumerate(self.words_i2w) )
        
        max_length = min(max([len(txt) for txt in texts]), 1024)
        
        self.length = max_length
       
        texts = [[w2i.get(w, w2i['__UNK__']) for w in txt] for txt in texts]
        self.texts = [ torch.tensor(txt[:max_length] + [0] * (max_length - len(txt))) for txt in texts ]

        assert(len(self.texts) == len(self.labels))
        
        print('Dataset подготовлен')
        
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

ds = Dataset('./20newsgroups')

KeyError: 'newsgroup'

In [ ]:
ds_train, ds_val = train_test_split(ds, test_size=0.1, random_state=7)

In [ ]:
def evaluate(net, ds):
    dl = torch.utils.data.DataLoader(ds, batch_size=128, shuffle=False)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.eval()
    total_loss = 0
    count = 0
    matches = 0
    for X, y in dl:
        count += len(y)
        preds = net(X)
        total_loss += float(loss(preds, y)) # очень важно преобразовать во float здесь, иначе утекает память !!!
        pred_classes = torch.argmax(preds, dim=1)
        matches += int(sum(pred_classes == y))
    return float(total_loss/count), float(matches/count), count


def train_and_test(net, ds_train, ds_val, optimizer_class=torch.optim.Adam, n_epochs=10, lr=0.01, report_on=-1, batch_size=1024):
    
    print('========================================================')
   
    params = [ p for p in net.parameters() if p.requires_grad]
    print('Число обучаемых параметров', len(params))
    optimizer = optimizer_class(params, lr=0.01)
    print(' Оптимизатор: ', optimizer)

    def print_results():
        for title, ds in [
#                ('Тренировочный', ds_train), # слишком долго для resnet 
                ('Валидационный', ds_val)
            ]:
            r = evaluate(net, ds)
            print(f"    {title} набор: кросс-энтропия: {r[0]:.2f}, доля совпадений: {r[1]*100:.1f}%")
        return r
    
    print(' До обучения:')
    print_results()
    
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()    

    net.train()
    report_on = max(len(dl_train) if report_on == -1 else report_on, 1)
    for epoch in range(n_epochs):
        print('--------- Эпоха ', epoch, '/', n_epochs, ' ------------------')
        running_loss = 0.0
        for i, data in enumerate(tqdm(dl_train)):
            inputs, labels = data[0], data[1]

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            if (i+1) % report_on == 0:    # печатаем каждые report_on mini-batches
                print('Выучено батчей : %5d; loss: %.3f' % (i + 1, running_loss / report_on))
                running_loss = 0.0
                
    print(' Обучение закончено!')
    print(' После обучения:')
    return print_results()

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim = 128, linear_dim = 256, kernel_size=3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv = nn.Conv1d(embedding_dim, linear_dim, kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(linear_dim, num_classes)
        
    def forward(self, x):        
        output = self.embedding(x)
        #                       B  F  L         
        output = output.permute(0, 2, 1)
        output = self.conv(output)
        output = self.relu(output)
        output = torch.max(output, axis=2).values
        output = self.linear(output)
        
        return output

In [ ]:
net = Net(20, len(ds.words_i2w))

train_and_test(net, ds_train, ds_val)

In [ ]:
# сохранение обученного net, остальное в цикле ниже

results = [{ 'embedding_dim': 128, 'linear_dim': 256, 'kernel_size': 3, 'net': net, 'result': evaluate(net, ds_val) }] 

In [ ]:
def create_and_test(embedding_dim=128, linear_dim=256, kernel_size=3):
    print('########################################################################')
    print(f"embedding_dim = {embedding_dim}, linear_dim = {linear_dim}, kernel_size = {kernel_size}")
    net = Net(20, len(ds.words_i2w), kernel_size=2)
    r = train_and_test(net, ds_train, ds_val)
    results.append({ 'embedding_dim': embedding_dim, 'linear_dim': linear_dim, 'kernel_size': kernel_size, 'net': net, 'result': r })    
    
    
for kernel_size in [2, 5, 7]:
    create_and_test(kernel_size=kernel_size)

for linear_dim in [128, 256, 512]:
    create_and_test(linear_dim=linear_dim)

for embedding_dim in [ 64, 256 ]:
    create_and_test(embedding_dim=embedding_dim)

In [ ]:
for r in tqdm(results): # ошибка в train_and_test была исправлена позже, результат не возвращался
    r['result'] = evaluate(r['net'], ds_val)

pd.DataFrame(results)

In [ ]:
create_and_test(embedding_dim=64, linear_dim=256, kernel_size=2)

In [ ]:
create_and_test(embedding_dim=64, linear_dim=256, kernel_size=7)

In [ ]:
df = pd.DataFrame(results)
df['loss'] = df['result'].apply(lambda r: r[0])
df['matches'] = df['result'].apply(lambda r: r[1])
df

In [ ]:
df.iloc[df['matches'].idxmax()]